In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from nltk.corpus import stopwords
from fuzzywuzzy import process, fuzz

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
import pandas as pd
from datetime import date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from datetime import datetime


from datetime import datetime,timedelta,date
from dateutil.parser import parse

In [ ]:
#df.disposition_raw.value_counts()

#### Layer 1 of charge type cleaning

#### Layer 2 charge type cleaning

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1NctDVIdWj5VLov0VgdH-FnHTAXnBYORl/edit?usp=sharing&ouid=102112814955431863841&rtpof=true&sd=true' # The shareable link
# fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':"1NctDVIdWj5VLov0VgdH-FnHTAXnBYORl"})
downloaded.GetContentFile('StateData.xls')
df = pd.read_excel('StateData.xls')

<built-in function id>


In [ ]:
# df = pd.concat(pd.read_excel('Missouri Data For Possible Expungement.xls', sheet_name=None), ignore_index=True)

In [ ]:
df.shape

(49816, 26)

In [ ]:
df.head

<bound method NDFrame.head of                                       SID                                 OCN  \
0      0x0028C4256970AA44051373D1AC5E7CDC  0xDE42B4104AF456311E818041A48A526F   
1      0x00323BAA4B3F4B2ACA92E15EB977F1D9  0x0E35781CA99320F355DFA1E09D82013B   
2      0x00323BAA4B3F4B2ACA92E15EB977F1D9  0x0E35781CA99320F355DFA1E09D82013B   
3      0x00323BAA4B3F4B2ACA92E15EB977F1D9  0x0E35781CA99320F355DFA1E09D82013B   
4      0x00323BAA4B3F4B2ACA92E15EB977F1D9  0x0E35781CA99320F355DFA1E09D82013B   
...                                   ...                                 ...   
49811  0xFFF311DDA8AD756510D4ECD415F69A24  0x4F06C4C0EDAED1E73923745FE3C39CDF   
49812  0xFFF311DDA8AD756510D4ECD415F69A24  0xD1F9452C31DACA20C5175DF753D89DB8   
49813  0xFFF33C95DA4A1EBCBCC42730B81F220E  0xAB3C3D7C08C7FF5886EB0BB979CE30D7   
49814  0xFFFBF762F49F67D89ED96AF69AE0E93C  0x446A4C4D61CC2ABE0A73E53FB43D59C6   
49815  0xFFFEB5D533F05250B5CE522673EB5D5A  0x7575F5A1649FBC4BE75127231E013FE3  

In [ ]:
df.SID.nunique()

5000

In [ ]:
df.columns

Index(['SID', 'OCN', 'DOA', 'AgeAtArrest', 'ACTION_DATE', 'RAC', 'SEX',
       'POB_CITY', 'POB_COUNTRY', 'CNT', 'UCC_OFF', 'CLASSIFICATION',
       'LONG_DESC', 'Charge_Type', 'ACTION', 'Sentence_TYPE', 'SENTENCE_YEAR',
       'SENTENCE_MONTH', 'SENTENCE_DAY', 'SUSP_IND', 'PROB_STATUS',
       'PROB_STAT_DATE', 'PROB_COMP_DATE', 'FINE_IND', 'FINE_SUSPND_IN',
       'COURT_COST_IND'],
      dtype='object')

In [ ]:
df = df[['SID','ACTION_DATE','CLASSIFICATION',
       'ACTION', 'Charge_Type',
       'LONG_DESC','RAC']]

In [ ]:
df.rename(columns={'SID':'id_raw', 'ACTION':'disposition_raw', 'CLASSIFICATION':'classification',
                   'Charge_Type':'charge_type_raw', 'LONG_DESC':'charge_raw',
                   'ACTION_DATE':'disposition_date_raw'
                  },inplace=True)


<ipython-input-14-cb1c9e63905a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'SID':'id_raw', 'ACTION':'disposition_raw', 'CLASSIFICATION':'classification',


In [ ]:
df.head(5)

,id_raw,disposition_date_raw,classification,disposition_raw,charge_type_raw,charge_raw,RAC
0,0x0028C4256970AA44051373D1AC5E7CDC,2019-05-10,NaN,418,L,SHOPLIFTING,B
1,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,F,DISTRIBUTE/DELIVER/MANUFACTURE/PRODUCE OR ATTE...,W
2,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,F,DISTRIBUTE/DELIVER/MANUFACTURE/PRODUCE OR ATTE...,W
3,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,F,DISTRIBUTE/DELIVER/MANUFACTURE/PRODUCE OR ATTE...,W
4,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,F,DISTRIBUTE/DELIVER/MANUFACTURE/PRODUCE OR ATTE...,W


## Data Cleaning

#### Date format change



In [ ]:
# Converting disposition_date to datetime
df['disposition_date_raw'] = pd.to_datetime(df['disposition_date_raw'],errors = 'coerce')

<ipython-input-16-a3fc8ee55e79>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['disposition_date_raw'] = pd.to_datetime(df['disposition_date_raw'],errors = 'coerce')


#### Dropping outlier records

In [ ]:
# Keep only 1992 to 2018 records
# Drop in number of records -
df = df[(df['disposition_date_raw'].dt.year>=1992) &(df['disposition_date_raw'].dt.year<=2018)]

In [ ]:
df.shape

(47717, 7)

In [ ]:
df.id_raw.nunique()

4747

In [ ]:
df.columns

Index(['id_raw', 'disposition_date_raw', 'classification', 'disposition_raw',
       'charge_type_raw', 'charge_raw', 'RAC'],
      dtype='object')

In [ ]:
"""
Function to easily assign "not guilty", "exclude" or "guilty" standardized values to our disposition field
"""
def normalize_disposition_new(flag_disposition):
    conv_terms = ["411","412","413","414","418"]
    conv_terms = "|".join(conv_terms)

    non_conv_terms = ["401","403","405","407"]
    non_conv_terms = "|".join(non_conv_terms)

    if re.search(conv_terms, flag_disposition):
        return "Conviction"
    elif re.search(non_conv_terms, flag_disposition):
        return "Nonconviction"
    else:
        return "Unknown"

#### Disposition Normalization

In [ ]:
##############################
# Disposition Normalization
##############################

print("Disposition Normalization :%s %s"%df.shape)
# Clean Disposition column (Strip, Lower)
df['disposition_raw'] = df['disposition_raw'].astype(str)
df['disposition_raw'] = df['disposition_raw'].str.strip('disposition_raw')
df['disposition_raw'] = df['disposition_raw'].str.lower()

df['disposition_normalization'] = df['disposition_raw'].apply(normalize_disposition_new)
print('Disposition Normalization Value Count Distribution: ')
print(df['disposition_normalization'].value_counts(normalize=True))
print('Disposition Normalization Value Counts: ')
print(df['disposition_normalization'].value_counts())

Disposition Normalization :47717 7
Disposition Normalization Value Count Distribution: 
Conviction       0.851269
Nonconviction    0.140705
Unknown          0.008026
Name: disposition_normalization, dtype: float64
Disposition Normalization Value Counts: 
Conviction       40620
Nonconviction     6714
Unknown            383
Name: disposition_normalization, dtype: int64


In [ ]:
#Function to normalized Charge Types
def normalize_charge_type(charge_type):
    """
    Normalize charge type values in the charge_type field to more consistent names
    """
    felony_charge_type_values = ["f1", "f2", "f3", "felony","f-1","f-2","f-3","f-4","f-5","felony"]
    felony_charge_type_terms = "|".join(felony_charge_type_values)

    misdemeanor_charge_type_values1 = ["non felony","non-felony","non-traffic"]
    misdemeanor_charge_type_terms1 = "|".join(misdemeanor_charge_type_values1)

    misdemeanor_charge_type_values2 = ["misd", "m1", "m2", "m3", "m4", "meanor","m-1","m-2","m-3","m-4","mean,"," mm"]
    misdemeanor_charge_type_terms2 = "|".join(misdemeanor_charge_type_values2)

    # traffic is too simple that it took a traffic misdemeanor and made it a felony; therefore, misdemeanor check in control flow first
    minor_charge_type_values = ["traffic", "petty", "infraction", "violation","municipal","county","minor off","non-crim","ordinance"]
    minor_charge_type_terms = "|".join(minor_charge_type_values)

    # we need an exact match on the letter m which means it's a misdemeanor
    if re.search(misdemeanor_charge_type_terms1, charge_type):
        return "misdemeanor"
    elif re.search(felony_charge_type_terms, charge_type) or charge_type=="f" or charge_type=="fel" or charge_type=="fe":
        return "felony"
    elif re.search(misdemeanor_charge_type_terms2, charge_type) or charge_type=="m" or charge_type=="m*" or charge_type=="mis"  :
        return "misdemeanor"
    elif re.search(minor_charge_type_terms, charge_type):
        return "minor"
    else:
        return "not classified"

##### Apply charge type Normalization

In [ ]:
##############################
# Charge Type Normalization
##############################
print("Charge Type Normalization :%s %s"%df.shape)
# Clean Charge Type Column - Convert to string and in lower case
df['charge_type_raw'] = df['charge_type_raw'].astype(str)
df['charge_type_raw'] = df['charge_type_raw'].str.lower()
df['charge_type_normalization'] = df['charge_type_raw'].apply(normalize_charge_type)
df['charge_type_normalized']=df['charge_type_normalization']
print('Charge Type Normalization Value Counts Distribution: ')
print(df['charge_type_normalized'].value_counts(normalize=True))
print('Charge Type Normalization Value Counts: ')
print(df['charge_type_normalized'].value_counts())

Charge Type Normalization :47717 8
Charge Type Normalization Value Counts Distribution: 
felony            0.754092
misdemeanor       0.194752
not classified    0.051156
Name: charge_type_normalized, dtype: float64
Charge Type Normalization Value Counts: 
felony            35983
misdemeanor        9293
not classified     2441
Name: charge_type_normalized, dtype: int64


In [ ]:
def charge_clearning(df):

    df.charge_raw = df.charge_raw.astype(str).apply(lambda x: x.replace('-',' '))
    print("Charge Type Normalization types....")
    print(df.charge_type_normalized.value_counts())
    df['charge_raw']=df['charge_raw'].str.lower().str.split()
    stop = stopwords.words('english')
    # Removing the stop words in Charge Column
    df['charge_raw']=df['charge_raw'].apply(lambda x: [item for item in x if item not in stop])
    df['charge_raw']=df['charge_raw'].str.join(" ")

    #Three list is created that contains charges with respect to the condition provided

    misdemeanor_list = df[df.charge_type_normalized == "misdemeanor"].charge_raw.unique().tolist()
    felony_list = df[df.charge_type_normalized == "felony"].charge_raw.unique().tolist()
    minor_list = df[df.charge_type_normalized == "minor"].charge_raw.unique().tolist()

    def classify(a,b):
        if a == 'not classified':
            if b in felony_list:
                return 'felony'
            elif b in misdemeanor_list:
                return 'misdemeanor'
            elif b in minor_list:
                return 'minor'
            else:
                return 'not classified'
        else:
            return a

    df['clean_charge'] = df[['charge_type_normalized','charge_raw']].astype(str).apply(lambda x: classify(x['charge_type_normalized'],x['charge_raw']),axis=1)

    #Copying the data to new dataframe to CLEAN it more
    classified_data_clean=df.copy()
    classified_data_clean.charge_type_normalized.value_counts()


    classified_data_clean['minor_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "minor" else 0)
    classified_data_clean['misdemeanor_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "misdemeanor" else 0)
    classified_data_clean['felony_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "felony" else 0)
    classified_data_clean['Unknown_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "not classified" else 0)

    #Fuzzy match classification

    misdemeanor_list2 = classified_data_clean[classified_data_clean.clean_charge == "misdemeanor"].charge_raw.unique().tolist()
    felony_list2 = classified_data_clean[classified_data_clean.clean_charge == "felony"].charge_raw.unique().tolist()
    minor_list2 = classified_data_clean[classified_data_clean.clean_charge == "minor"].charge_raw.unique().tolist()

    def Fel_fuzzy_match(a,b):
        if a == 'not classified':
            for f in felony_list:
                #fuzz_ratio = fuzz.ratio
                if ((fuzz.partial_ratio(b,f))>=50):
                    return 'felony'
                else:
                    return 'not classified'
        else:
            return a

    classified_data_clean['fuzzy_f']=classified_data_clean[['clean_charge','charge_raw']].astype(str).apply(lambda x: Fel_fuzzy_match(x['clean_charge'],x['charge_raw']),axis=1)

    def misd_fuzzy_match(a,b):
        if a == 'not classified':
            for m in misdemeanor_list:
                #fuzz_ratio = fuzz.ratio
                if ((fuzz.partial_ratio(b,m))>=30):
                    return 'misdemeanor'
                else:
                    return 'not classified'
        else:
            return a

    classified_data_clean['fuzzy_md']=classified_data_clean[['fuzzy_f','charge_raw']].astype(str).apply(lambda x: misd_fuzzy_match(x['fuzzy_f'],x['charge_raw']),axis=1)

    def min_fuzzy_match(a,b):
        if a == 'not classified':
            for n in minor_list2:
                #fuzz_ratio = fuzz.ratio
                if ((fuzz.partial_ratio(b,n))>=30):
                    return 'minor'
                else:
                    return 'not classified'
        else:
            return a

    classified_data_clean['fuzzy_min']=classified_data_clean[['fuzzy_md','charge_raw']].astype(str).apply(lambda x: min_fuzzy_match(x['fuzzy_md'],x['charge_raw']),axis=1)

    print("Value count on classified_data_clean.fuzzy_min ")
    print("classified_data_clean.fuzzy_min")

    classified_data_clean['minor_flag']=classified_data_clean['fuzzy_min'].apply(lambda x: 1 if x == "minor" else 0)
    classified_data_clean['misdemeanor_flag']=classified_data_clean['fuzzy_min'].apply(lambda x: 1 if x == "misdemeanor" else 0)
    classified_data_clean['felony_flag']=classified_data_clean['fuzzy_min'].apply(lambda x: 1 if x == "felony" else 0)
    classified_data_clean['Unknown_flag']=classified_data_clean['fuzzy_min'].apply(lambda x: 1 if x == "not classified" else 0)
    #df['NonconvictionDivDef_flag']=df['disposition_normalization'].apply(lambda x: 1 if x == "NonconvictionDivDef" else 0)

    def charge_clean2(data,data1):
        if data1 == 'not classified':
            if re.search('driveway', data):
                return "minor"
            elif re.search('parking', data):
                return "minor"
            elif re.search('drivers license', data):
                return "minor"
            elif re.search('driving', data):
                return "minor"
            elif re.search('safety belt', data):
                return "minor"
            elif re.search('seat belt', data):
                return "minor"
            elif re.search('seatbelt', data):
                return "minor"
            elif re.search('speeding', data):
                return "minor"
            else:
                return 'not classified'
        else:
            return data1

    classified_data_clean['clean_charge'] = classified_data_clean[['fuzzy_min','charge_raw']].astype(str).apply(lambda x: charge_clean2(x['charge_raw'],x['fuzzy_min']),axis=1)

    classified_data_clean['minor_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "minor" else 0)
    classified_data_clean['misdemeanor_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "misdemeanor" else 0)
    classified_data_clean['felony_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "felony" else 0)
    classified_data_clean['Unknown_flag']=classified_data_clean['clean_charge'].apply(lambda x: 1 if x == "not classified" else 0)
    #df['NonconvictionDivDef_flag']=df['disposition_normalization'].apply(lambda x: 1 if x == "NonconvictionDivDef" else 0)


    df2 = classified_data_clean.drop(columns=['minor_flag','misdemeanor_flag', 'felony_flag', 'Unknown_flag', 'fuzzy_f','fuzzy_md','fuzzy_min'])

    df2.drop(['charge_type_normalized'], axis=1, inplace=True)

    df2.rename(columns = {'clean_charge':'charge_type_normalized'},inplace = True)

    df2["charge_type_normalization"] = df2["charge_type_normalized"]   # To avoid ambiguity
    print("Charge Type Normalization types.... Post cleaning")
    print(df2.charge_type_normalization.value_counts())

    return df2

In [ ]:
df.shape

(47717, 10)

In [ ]:
df = charge_clearning(df).copy()

Charge Type Normalization types....
felony            35983
misdemeanor        9293
not classified     2441
Name: charge_type_normalized, dtype: int64
Value count on classified_data_clean.fuzzy_min 
classified_data_clean.fuzzy_min
Charge Type Normalization types.... Post cleaning
felony         36738
misdemeanor    10561
None             418
Name: charge_type_normalization, dtype: int64


In [ ]:
df.shape

(47717, 10)

In [ ]:
# Creatting dummies
df=pd.get_dummies(df, columns=['charge_type_normalization'])

In [ ]:
#If the charge type is not present create a NAN column so that at later stages you wont' get error
chargelst = df.columns
if 'charge_type_normalization_felony' not in chargelst:
    df['charge_type_normalization_felony'] = np.NaN
if 'charge_type_normalization_misdemeanor' not in chargelst:
    df['charge_type_normalization_misdemeanor'] = np.NaN
if 'charge_type_normalization_not classified' not in chargelst:
    df['charge_type_normalization_not classified'] = np.NaN
if 'charge_type_normalization_minor' not in chargelst:
    df['charge_type_normalization_minor'] = np.NaN

#### Baseline filters

### Run from this cell for analysis



In [ ]:
convictions_mis_results# Create a new column called convictd that has a value of 1 if someone is guilty and a 0 otherwise.
df['convicted'] = np.where(df['disposition_normalization']=="Conviction", 1, 0)
# Create conviction type columns classifying charges as- 'Guilty', 'Not Guilty' and 'Unresolved'.
df['conviction_type']='Unresolved'

In [ ]:
# Convictions

# Create a new column called convictd that has a value of 1 if someone is guilty and a 0 otherwise.
df['convicted'] = np.where(df['disposition_normalization']=="Conviction", 1, 0)
# Create conviction type columns classifying charges as- 'Guilty', 'Not Guilty' and 'Unresolved'.
df.loc[(df['disposition_normalization'] == 'Conviction'), 'conviction_type'] = 'guilty'
df.loc[(df['disposition_normalization'] == 'Nonconviction'), 'conviction_type'] = 'not_guilty'
df=pd.get_dummies(df,columns=['conviction_type'])

In [ ]:
# Additional filter - On disposition_normalization
df = df[df['disposition_normalization'].isin(["Conviction", "Nonconviction"])]

In [ ]:
print("Number of people after additional filter:")
print(df.id_raw.nunique())

Number of people after additional filter:
4685


In [ ]:
mo_df = df.copy()

In [ ]:
# prepare date stuff
mo_df["datetime"] = pd.to_datetime(mo_df['disposition_date_raw'], infer_datetime_format=True)
current_date = pd.to_datetime("2021-02-01", infer_datetime_format=True)
three_year_delta = pd.Timedelta("1095 days")
seven_year_delta = pd.Timedelta("2555 days")
ten_year_delta = pd.Timedelta("3650 days")
one_year_delta =  pd.Timedelta("365 days")

In [ ]:
 # find people with subsequent convictions
print(len(mo_df.index))
convictions_df = mo_df[mo_df["disposition_normalization"].str.match('Conviction',na=False)]
print(len(convictions_df.index))
mis_df = mo_df[mo_df["charge_type_normalized"].str.match('misdemeanor',na=False)]
print(len(mis_df.index))
fel_df = mo_df[mo_df["charge_type_normalized"].str.match('felony',na=False)]
print(len(fel_df.index))
intox_df = mo_df[mo_df["charge_raw"].str.contains('intox',na=False)]
print(len(intox_df.index))

47334
40620
10494
36423
494


In [ ]:
## Below cell not working (CG)

In [ ]:
user_list = convictions_df.id_raw.unique()
multiple_convictions_d = {}
for user in user_list:
    temp_df = convictions_df[convictions_df["id_raw"] == user]
    date_list = temp_df["disposition_date_raw"].unique()
    if len(date_list) > 1:
        multiple_convictions_d[user] = 1

In [ ]:
user_list = mis_df.id_raw.unique()
multiple_mis_d = {}
for user in user_list:
    temp_df = mis_df[mis_df["id_raw"]== user]
    date_list = temp_df["disposition_date_raw"].unique()
    if len(date_list) > 2:
        multiple_mis_d[user] = 1

In [ ]:
user_list = fel_df.id_raw.unique()
multiple_fel_d = {}
for user in user_list:
    temp_df = fel_df[fel_df["id_raw"]== user]
    date_list = temp_df["disposition_date_raw"].unique()
    if len(date_list) > 1:
        multiple_fel_d[user] = 1

In [ ]:
pending_d = {}
for user in user_list:
    temp_df = mo_df[mo_df["id_raw"]== user]
    pending_list = temp_df["disposition_raw"].unique()
    if "case pending" in pending_list:
        pending_d[user] = 1

In [ ]:
print(len(multiple_convictions_d))
print(len(multiple_mis_d))
print(len(multiple_fel_d))
print(len(user_list))

1026
157
766
2592


In [ ]:
print(mo_df["charge_type_raw"].unique())

['f' 'm' 'l' 'nan' 'i' 'u']


In [ ]:
print(mo_df.charge_type_normalized.unique())

['felony' 'misdemeanor' 'None']


In [ ]:
print(mo_df["disposition_raw"].unique())

['411' '418' '407' '405' '401' '403']


In [ ]:
print(mo_df["disposition_normalization"].unique())

['Conviction' 'Nonconviction']


In [ ]:
mo_df.charge_raw.value_counts()

forgery                                                                                      6833
statutory sodomy 2nd degree                                                                  4430
statutory rape 2nd degree                                                                    3196
possession controlled substance except 35 grams less marijuana                               2761
dwi alcohol                                                                                  1568
                                                                                             ... 
assault 3rd degree 3rd subsequent offense family/household member                               1
livestock captive wildlife theft excess $3,000 value prior offender                             1
tampering property another 2nd degree                                                           1
statutory sodomy attempted statutory sodomy deviate sexual intercourse person less 14 yoa       1
excessive noise vehi

In [ ]:
def not_eligible_Section_610_140(row):
    charge = row["charge_raw"].lower()
    charge_type = row["charge_type_raw"]
    charge_type_normalized = row["charge_type_normalized"].lower()
    disposition_normalization = row["disposition_normalization"].lower()
    if "arson 1st" in charge:
        return True
    if "assault 1st" in charge:
        return True
    if "forcible rape" in charge:
        return True
    if "statutory rape 1st" in charge:
        return True
    if "attempted rape 1st" in charge:
        return True
    if "statutory sodomy 1st" in charge:
        return True
    if "forcible sodomy" in charge:
        return True
    if "attempted sodomy 1st" in charge:
        return True
    if "assault 2nd" in charge and "domestic" not in charge:
        return True
    if "kidnapping 1st" in charge:
        return True
    if "murder 2nd" in charge:
        return True
    if "elder abuse" in charge:
        return True
    if "568.060" in charge:
        return True
    if "child molestation" in charge:
        return True
    if "robbery 1st" in charge and "atmp" not in charge:
        return True
    if "armed" in charge:
        return True
    if "felony a" in charge_type:
        return True
    if "566.149" in charge:
        return True
    if "566.150" in charge:
        return True
    if "sex offender present/loiter within 500 feet park playground/pool 1st offense" in charge:
        return True
    if "death" in charge and "felony" in charge_type_normalized:
        return True
    if "assault" in charge and "felony" in charge_type_normalized:
        return True
    if "domestic" in charge and "felony" in charge_type_normalized:
        return True
    if "domestic" in charge and "misdemeanor" in charge_type_normalized:
        return True
    if "kidnapping" in charge and "felony" in charge_type_normalized:
        return True
    if "alcohol" in charge and "conviction" in disposition_normalization:
        return True
    return False

In [ ]:
def convictions_mis(row):
    idx = row["id_raw"]
    if idx in multiple_mis_d or idx in multiple_fel_d:
        return False
    after_waiting_period = row["datetime"] + one_year_delta
    if current_date > after_waiting_period:
        return True
    return False

In [ ]:
def convictions_mis_traffic(row):
    idx = row["id_raw"]
    if idx in intox_df:
        after_waiting_period = row["datetime"] + ten_year_delta
        if current_date > after_waiting_period:
            return True
    return False

In [ ]:
def convictions_fel(row):
    idx = row["id_raw"]
    if idx in multiple_mis_d or idx in multiple_fel_d:
        return False
    after_waiting_period = row["datetime"] + three_year_delta
    if current_date > after_waiting_period:
        return True
    return False

In [ ]:
def non_convictions(row):
    idx = row["id_raw"]
    return True

In [ ]:
def apply_function(row):
    conviction = row["disposition_normalization"]
    idx = row["id_raw"]
    if idx in pending_d:
        return "Not Eligible"
    if conviction == "Conviction":
        not_eligible_Section_610_140_results = not_eligible_Section_610_140(row)
        convictions_mis_results = convictions_mis(row)
        convictions_mis_traffic_results = convictions_mis_traffic(row)
        convictions_fel_results = convictions_fel(row)
        if not_eligible_Section_610_140_results == True:
            return "Not Eligible"
        if convictions_mis_results == True:
            return "Conviction Eligible"
        if convictions_mis_traffic_results == True:
            return "Conviction Eligible"
        if convictions_fel_results == True:
            return "Conviction Eligible"
    if conviction == "Nonconviction":
        non_convictions_results = non_convictions(row)
        if non_convictions_results == True:
            return "Nonconviction Eligible"
    return "Not Eligible"

In [ ]:
mo_df["Eligibility"] = mo_df.apply(apply_function, axis=1)

In [ ]:
print(mo_df.Eligibility.value_counts())

Not Eligible              32156
Conviction Eligible        8464
Nonconviction Eligible     6714
Name: Eligibility, dtype: int64


### Numbers

In [ ]:
num_people = mo_df.id_raw.nunique()
print("Number of people in sample: ",num_people)

Number of people in sample:  4685


In [ ]:
convictions_df = mo_df[(mo_df["disposition_normalization"] == "Conviction")]
num_conv = convictions_df.id_raw.nunique()
print("Number of people with convictions in sample:",num_conv)

Number of people with convictions in sample: 3836


In [ ]:
convictions_df.head(5)

,id_raw,disposition_date_raw,classification,disposition_raw,charge_type_raw,charge_raw,RAC,disposition_normalization,charge_type_normalized,charge_type_normalization_None,charge_type_normalization_felony,charge_type_normalization_misdemeanor,charge_type_normalization_not classified,charge_type_normalization_minor,convicted,conviction_type_Unresolved,conviction_type_guilty,conviction_type_not_guilty,datetime,Eligibility
1,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,f,distribute/deliver/manufacture/produce attempt...,W,Conviction,felony,0,1,0,NaN,NaN,1,0,1,0,2013-10-08,Not Eligible
2,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,f,distribute/deliver/manufacture/produce attempt...,W,Conviction,felony,0,1,0,NaN,NaN,1,0,1,0,2013-10-08,Not Eligible
3,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,f,distribute/deliver/manufacture/produce attempt...,W,Conviction,felony,0,1,0,NaN,NaN,1,0,1,0,2013-10-08,Not Eligible
4,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,f,distribute/deliver/manufacture/produce attempt...,W,Conviction,felony,0,1,0,NaN,NaN,1,0,1,0,2013-10-08,Not Eligible
5,0x00323BAA4B3F4B2ACA92E15EB977F1D9,2013-10-08,B,411,f,distribute/deliver/manufacture/produce attempt...,W,Conviction,felony,0,1,0,NaN,NaN,1,0,1,0,2013-10-08,Not Eligible


In [ ]:
convictions_df.groupby('RAC').nunique()['id_raw']

RAC
A       2
B    1271
I       5
U      11
W    2560
Name: id_raw, dtype: int64

In [ ]:
convictions_df['RAC'].unique()

array(['W', 'B', 'U', 'A', 'I', nan], dtype=object)

In [ ]:
fel_conv_df = convictions_df[(convictions_df["charge_type_normalized"] == "felony")]
num_fel_conv = fel_conv_df.id_raw.nunique()
print("Number of people with felony convictions in sample:",num_fel_conv)

Number of people with felony convictions in sample: 2007


In [ ]:
fel_conv_df.groupby('RAC').nunique()['id_raw']

RAC
A       1
B     805
I       3
U       5
W    1199
Name: id_raw, dtype: int64

In [ ]:
new_df = pd.DataFrame(fel_conv_df.groupby('id_raw').nunique()['RAC'] > 1)
new_df[new_df.RAC == True]

,RAC
id_raw,
0x64AB3FABD982A13A986FB0335B018ECA,True
0xDEFF7023AB39ECEC4901091A7C4AA046,True
0xEB1FD8C3E33B921F942826A5FF7F14AB,True
0xEE65F145053090994010495336C6CAEF,True
0xF80B9D0148FB9DDC28E2605854802F03,True


In [ ]:
22539-69

22470

In [ ]:
fel_conv_df[fel_conv_df['id_raw'] == '0x0155510AD20EE4CB897C1FF7A41F0E7E']

,id_raw,disposition_date_raw,classification,disposition_raw,charge_type_raw,charge_raw,RAC,disposition_normalization,charge_type_normalized,charge_type_normalization_None,charge_type_normalization_felony,charge_type_normalization_misdemeanor,charge_type_normalization_not classified,charge_type_normalization_minor,convicted,conviction_type_Unresolved,conviction_type_guilty,conviction_type_not_guilty,datetime,Eligibility


In [ ]:
mis_conv_df = convictions_df[(convictions_df["charge_type_normalized"] == "misdemeanor")]
num_mis_conv = mis_conv_df.id_raw.nunique()
print("Number of people with misdemeanor convictions in sample:",num_mis_conv)

Number of people with misdemeanor convictions in sample: 2312


In [ ]:
fel_charge_df = mo_df[(mo_df["charge_type_normalized"] == "felony")]
num_fel_charge = fel_charge_df.id_raw.nunique()
print("Number of people with felony charges in sample:",num_fel_charge)

Number of people with felony charges in sample: 2592


In [ ]:
eli_df = mo_df[(mo_df["Eligibility"] == "Conviction Eligible")]
num_eligible = eli_df.id_raw.nunique()
print("Number of people with eligible conviction charges in sample:",num_eligible)

In [ ]:
eli_df.groupby('RAC').nunique()['id_raw']

In [ ]:
both_eli_df = mo_df[(mo_df["Eligibility"] == "Conviction Eligible") | (mo_df['Eligibility'] == "Nonconviction Eligible")]
num_both_eligible = both_eli_df.id_raw.nunique()
print("Number of people with eligible conviction & nonconviction charges in sample:",num_both_eligible)

In [ ]:
people_list = mo_df.id_raw.unique()
count_all_convictions_eligible = 0
count_partial_convictions_eligible = 0

for person in people_list:
    temp_df = mo_df[mo_df["id_raw"] == person]
    temp2_df = temp_df[temp_df["disposition_normalization"]=="Conviction"]
    eligibility_results = temp2_df["Eligibility"].unique()
    if "Conviction Eligible" in eligibility_results and "Not Eligible" not in eligibility_results:
        count_all_convictions_eligible += 1
    elif "Conviction Eligible" in eligibility_results:
        count_partial_convictions_eligible += 1

print("num people with all convictions eligible:",count_all_convictions_eligible)
print("num people with partial convictions eligible:",count_partial_convictions_eligible)

In [ ]:
new_list = list()
for i in range(1,10):
    new_list.append(i)
new_list

In [ ]:
people_list = mo_df.id_raw.unique()
count_all_convictions_eligible = 0
# count_partial_convictions_eligible = 0
count_all_convictions_eligible_list = list()

for person in people_list:
    temp_df = mo_df[mo_df["id_raw"] == person]
    temp2_df = temp_df[temp_df["disposition_normalization"]=="Conviction"]
    eligibility_results = temp2_df["Eligibility"].unique()
    if "Conviction Eligible" in eligibility_results and "Not Eligible" not in eligibility_results:
        count_all_convictions_eligible += 1
        count_all_convictions_eligible_list.append(person)
#     elif "Conviction Eligible" in eligibility_results:
#         count_partial_convictions_eligible += 1

print("num people with all convictions eligible:",count_all_convictions_eligible)
# print("num people with partial convictions eligible:",count_partial_convictions_eligible)

In [ ]:
count_all_convictions_eligible_df = pd.DataFrame(count_all_convictions_eligible_list,columns=['id_raw'])
count_all_convictions_eligible_df

In [ ]:
count_all_convictions_eligible_df.id_raw.nunique()

In [ ]:
h71 = pd.merge(count_all_convictions_eligible_df,mo_df, on='id_raw', how ='inner')
h71.id_raw.nunique()

In [ ]:
h71.groupby('RAC').nunique()['id_raw']

In [ ]:
people_list = mo_df.id_raw.unique()
count_all_eligible = 0
count_partial_eligible = 0

for person in people_list:
    temp_df = mo_df[mo_df["id_raw"] == person]
    eligibility_results = temp_df["Eligibility"].unique()
    if "Not Eligible" not in eligibility_results:
        count_all_eligible += 1
    elif "Conviction Eligible" in eligibility_results or "Nonconviction Eligible" in eligibility_results:
        count_partial_eligible += 1

print("num people with all eligible:",count_all_eligible)
print("num people with partial eligible:",count_partial_eligible)

In [ ]:
people_list = mo_df.id_raw.unique()
count_fel_eligible = 0

for person in people_list:
    temp_df = mo_df[mo_df["id_raw"] == person]
    temp2_df = temp_df[temp_df["charge_type_normalized"]=="felony"]
    eligibility_results = temp2_df["Eligibility"].unique()
    if "Conviction Eligible" in eligibility_results:
        count_fel_eligible += 1

print("num people with felony convictions eligible:",count_fel_eligible)

In [ ]:
people_list = mo_df.id_raw.unique()
count_fel_eligible = 0
count_fel_eligible_list = list()

for person in people_list:
    temp_df = mo_df[mo_df["id_raw"] == person]
    temp2_df = temp_df[temp_df["charge_type_normalized"]=="felony"]
    eligibility_results = temp2_df["Eligibility"].unique()
    if "Conviction Eligible" in eligibility_results:
        count_fel_eligible += 1
        count_fel_eligible_list.append(person)

print("num people with felony convictions eligible:",count_fel_eligible)

In [ ]:
count_fel_eligible_df = pd.DataFrame(count_fel_eligible_list,columns=['id_raw'])
count_fel_eligible_df.head(5)

In [ ]:
count_fel_eligible_df.id_raw.nunique()

In [ ]:
h73 = pd.merge(count_fel_eligible_df,mo_df, on='id_raw', how ='inner')
h73.id_raw.nunique()

In [ ]:
h73.groupby('RAC').nunique()['id_raw']

### Top 10

In [ ]:
print(mo_df.charge_raw.value_counts())

In [ ]:
print(len(mo_df.index))

In [ ]:
print(both_eli_df.charge_raw.value_counts())

In [ ]:
print(len(both_eli_df.index))

In [ ]:
df.head()

In [ ]:
df[(df.charge_type_raw=="F") & (df.classification=="A")].charge_raw.drop_duplicates().to_csv("MO_ClassA_Felonies.csv")